In [1]:
import duckdb

In [2]:
# Query 1: Create E_COMMERCE_TRANSACTIONS table from CSV file '../data/e_commerce_transactions.csv'
duckdb.sql("DROP TABLE IF EXISTS E_COMMERCE_TRANSACTIONS")
duckdb.sql("CREATE TABLE E_COMMERCE_TRANSACTIONS AS SELECT * FROM read_csv_auto('../data/e_commerce_transactions.csv')")

In [3]:
# Query 2: Display first 5 rows of E_COMMERCE_TRANSACTIONS table
duckdb.sql("SELECT * FROM E_COMMERCE_TRANSACTIONS LIMIT 5")

┌──────────┬─────────────┬────────────┬───────────────┬────────────┬─────────────┐
│ order_id │ customer_id │ order_date │ payment_value │ decoy_flag │ decoy_noise │
│  int64   │    int64    │    date    │    double     │  varchar   │   double    │
├──────────┼─────────────┼────────────┼───────────────┼────────────┼─────────────┤
│   101000 │         684 │ 2024-01-29 │         38.93 │ A          │       39.51 │
│   101001 │         207 │ 2024-06-29 │         59.52 │ B          │       38.97 │
│   101002 │         288 │ 2024-03-20 │        355.52 │ B          │      338.14 │
│   101003 │         571 │ 2024-02-24 │        157.65 │ C          │      121.69 │
│   101004 │         871 │ 2024-01-11 │        287.68 │ C          │      254.92 │
└──────────┴─────────────┴────────────┴───────────────┴────────────┴─────────────┘

In [5]:
# Query 3: Check if we have null data in E_COMMERCE_TRANSACTIONS table
duckdb.sql("""SELECT COUNT(*) AS NULL_DATA
           FROM E_COMMERCE_TRANSACTIONS
           WHERE order_id IS NULL 
           OR customer_id IS NULL
           OR order_date IS NULL
           OR payment_value IS NULL
           OR decoy_flag IS NULL
           OR decoy_noise IS NULL""")

┌───────────┐
│ NULL_DATA │
│   int64   │
├───────────┤
│         0 │
└───────────┘

In [6]:
# Query 4: Check how many unique order_id in E_COMMERCE_TRANSACTIONS table
# to make sure we have no duplicates
duckdb.sql("SELECT COUNT(DISTINCT order_id) AS UNIQUE_ORDER_ID FROM E_COMMERCE_TRANSACTIONS")

┌─────────────────┐
│ UNIQUE_ORDER_ID │
│      int64      │
├─────────────────┤
│           10000 │
└─────────────────┘

In [13]:
# Query 5: Check how many unique customer_id in E_COMMERCE_TRANSACTIONS table
duckdb.sql("SELECT COUNT(DISTINCT customer_id) AS UNIQUE_CUSTOMER_ID FROM E_COMMERCE_TRANSACTIONS")

┌────────────────────┐
│ UNIQUE_CUSTOMER_ID │
│       int64        │
├────────────────────┤
│               1000 │
└────────────────────┘

In [14]:
# Query 6: Check minimum and maximum payment_value in E_COMMERCE_TRANSACTIONS table
duckdb.sql("""SELECT 
                MIN(payment_value) AS MIN_PAYMENT_VALUE, 
                MAX(payment_value) AS MAX_PAYMENT_VALUE 
              FROM E_COMMERCE_TRANSACTIONS""")

┌───────────────────┬───────────────────┐
│ MIN_PAYMENT_VALUE │ MAX_PAYMENT_VALUE │
│      double       │      double       │
├───────────────────┼───────────────────┤
│              0.55 │           1369.47 │
└───────────────────┴───────────────────┘

In [15]:
# Query 7: Check unique value from decoy_flag column in E_COMMERCE_TRANSACTIONS table
duckdb.sql("SELECT DISTINCT decoy_flag FROM E_COMMERCE_TRANSACTIONS")

┌────────────┐
│ decoy_flag │
│  varchar   │
├────────────┤
│ A          │
│ C          │
│ D          │
│ B          │
└────────────┘

In [16]:
# Query 8: Check minimum and maximum decoy_noise in E_COMMERCE_TRANSACTIONS table
duckdb.sql("""SELECT 
                MIN(decoy_noise) AS MIN_DECOY_NOISE, 
                MAX(decoy_noise) AS MAX_DECOY_NOISE 
              FROM E_COMMERCE_TRANSACTIONS""")

┌─────────────────┬─────────────────┐
│ MIN_DECOY_NOISE │ MAX_DECOY_NOISE │
│     double      │     double      │
├─────────────────┼─────────────────┤
│          -48.34 │         1468.46 │
└─────────────────┴─────────────────┘